# 02 - Data from the Web

## Deadline
Wednesday October 25, 2017 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.

## Background
In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need!
You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1) to understand quickly how to use it.

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fÃ©dÃ©rale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.

##  textual description of your thought process, the assumptions you made, and the solution you plan to implement!

### Thought process:

www.topuniversities.com 
1. get https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508228964836
    a. this is a json file
2. obtain name rank country region
3. get https://www.topuniversities.com/universities/ecole-polytechnique-f%C3%A9d%C3%A9rale-de-lausanne-epfl
    a. this is a html file
4. get number of faculty -- international and total, number of students -- international and total


### Our assumptions:


### Implementation solution:

In [36]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [37]:
req1 = requests.get('https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508228964836')
# req1.json()
body = req1.json()
body_data = body['data']

In [40]:
rank_list = []
df = pd.DataFrame(columns = ['Name', 'Rank', 'Country', 'Region', 'No. of International Faculty', 'No. of Total Faculty', 'No. of International Students', 'No. of Total Students'])

for i in range(200):
    name = body_data[i]['title']
    rank = body_data[i]['rank_display']
    country = body_data[i]['country']
    region = body_data[i]['region']
    url = body_data[i]['url']
    
    details = requests.get('https://www.topuniversities.com' + url)
    html_body = details.text
    soup = BeautifulSoup(html_body, 'html.parser')
    
    try:
        total_faculty = float(soup.find(class_='total faculty').find(class_='number').string[1:-1].replace(",", ""))
    except:
        total_faculty = float("NaN")
        print("Error in total faculty", name)
    
    try:
        international_faculty = float(soup.find(class_ = 'inter faculty').find(class_ = 'number').string[1:-1].replace(",", ""))
    except:
        international_faculty = float("NaN")
        print("Error in international faculty", name)
    try:
        total_students = float(soup.find(class_ = 'total student').find(class_ = 'number').string[1:-1].replace(",", ""))
    except:
        total_students = float("NaN")
        print("Error in total students", name)
    try:
        international_students = float(soup.find(class_ = 'total inter').find(class_ = 'number').string[1:-1].replace(",", ""))
    except:
        international_students = float("NaN")
        print("Error in international students", name)
    rank_list.append({'Name': name,
                      'Rank': rank,
                      'Country': country,
                      'Region': region,
                      'No. of International Faculty': international_faculty,
                      'No. of Total Faculty': total_faculty,
                      'No. of International Students': international_students,
                      'No. of Total Students': total_students})
    
top_ranking_universities = pd.DataFrame.from_dict(rank_list)
top_ranking_universities.head()

Error in total faculty New York University (NYU)
Error in international faculty New York University (NYU)
Error in total students New York University (NYU)
Error in international students New York University (NYU)
Error in international faculty Indian Institute of Science (IISc) Bangalore


,Country,Name,No. of International Faculty,No. of International Students,No. of Total Faculty,No. of Total Students,Rank,Region
0,United States,Massachusetts Institute of Technology (MIT),1679.0,3717.0,2982.0,11067.0,1,North America
1,United States,Stanford University,2042.0,3611.0,4285.0,15878.0,2,North America
2,United States,Harvard University,1311.0,5266.0,4350.0,22429.0,3,North America
3,United States,California Institute of Technology (Caltech),350.0,647.0,953.0,2255.0,4,North America
4,United Kingdom,University of Cambridge,2278.0,6699.0,5490.0,18770.0,5,Europe


In [41]:
top_ranking_universities.to_csv('top_ranking_universities.csv')